In [ ]:
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# PyTorch Training
This notebook trains a model to predict whether the given sonar signals are bouncing off a metal cylinder or off a cylindrical rock from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+%28Sonar%2C+Mines+vs.+Rocks%29).

## The data
The Sonar Signals dataset that this sample uses for training is provided by the UC Irvine Machine Learning Repository. Google has hosted the data on a public GCS bucket `gs://cloud-samples-data/ml-engine/sonar/sonar.all-data`.

* `sonar.all-data` is split for both training and evaluation

Note: Your typical development process with your own data would require you to upload your data to GCS so that you can access that data from inside your notebook. However, in this case, Google has put the data on GCS to avoid the steps of having you download the data from UC Irvine and then upload the data to GCS.

### Disclaimer
This dataset is provided by a third party. Google provides no representation, warranty, or other guarantees about the validity or any other aspects of this dataset.

# Build your model
First, you'll create the model (provided below). This is similar to your normal process for creating a PyTorch model. However, there is one key difference:

Downloading the data from GCS at the start of your file, so that you can access the data.

In [ ]:
from google.cloud import storage
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

In [ ]:
# ---------------------------------------
# Add code to download the data from GCS (in this case, using the publicly hosted data).
# you will then be able to use the data when training your model.
# ---------------------------------------
# Public bucket holding the census data
bucket = storage.Client().bucket('cloud-samples-data')

# Path to the data inside the public bucket
blob = bucket.blob('ml-engine/sonar/sonar.all-data')
# Download the data
blob.download_to_filename('sonar.all-data')

In [ ]:
# ---------------------------------------
# Read in the data
# ---------------------------------------

# Define the dataset to be used by PyTorch
class SonarDataset(Dataset):
    def __init__(self, csv_file):
        self.dataframe = pd.read_csv(csv_file, header=None)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # When iterating through the dataset get the features and targets
        features = self.dataframe.iloc[idx, :-1].values.astype(dtype='float64')

        # Convert the targets to binary values:
        # R = rock --> 0
        # M = mine --> 1
        target = self.dataframe.iloc[idx, -1:].values
        if target[0] == 'R':
            target[0] = 0
        elif target[0] == 'M':
            target[0] = 1
        target = target.astype(dtype='float64')

        # Load the data as a tensor
        data = {'features': torch.from_numpy(features),
                'target': target}
        return data


# Load the data
sonar_dataset = SonarDataset('./sonar.all-data')
# Create indices for the split
dataset_size = len(sonar_dataset)
test_size = int(0.2 * dataset_size)  # Use a test_split of 0.2
train_size = dataset_size - test_size

# Split the dataset
train_dataset, test_dataset = random_split(sonar_dataset,
                                           [train_size, test_size])
# Create our Dataloaders for training and test data
train_loader = DataLoader(
    train_dataset.dataset,
    batch_size=4,
    shuffle=True)
test_loader = DataLoader(
    test_dataset.dataset,
    batch_size=4,
    shuffle=True)

In [ ]:
# ---------------------------------------
# This is where your model code would go. Below is an example model using the census dataset.
# ---------------------------------------
torch.manual_seed(42)

# Create the Deep Neural Network
class SonarDNN(nn.Module):
    def __init__(self):
        super(SonarDNN, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(60, 60),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(60, 30),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(30, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# Create the model
net = SonarDNN().double()
optimizer = optim.SGD(net.parameters(),
                      lr=0.01,
                      momentum=0.5,
                      nesterov=False)

In [ ]:
# ---------------------------------------
# Define the training loop
# ---------------------------------------
def train(net, train_loader, optimizer, epoch):
    """Create the training loop"""
    net.train()
    criterion = nn.BCELoss()
    running_loss = 0.0

    for batch_index, data in enumerate(train_loader):
        features = data['features']
        target = data['target']

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(features)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if batch_index % 6 == 5:  # print every 6 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch, batch_index + 1, running_loss / 6))
            running_loss = 0.0

In [ ]:
# ---------------------------------------
# Define the testing loop
# ---------------------------------------
def test(net, test_loader):
    """Test the DNN"""
    net.eval()
    criterion = nn.BCELoss()  # https://pytorch.org/docs/stable/nn.html#bceloss
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            features = data['features']
            target = data['target']
            output = net(features)
            # Binarize the output
            pred = output.apply_(lambda x: 0.0 if x < 0.5 else 1.0)
            test_loss += criterion(output, target)  # sum up batch loss
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set:\n\tAverage loss: {:.4f}'.format(test_loss))
    print('\tAccuracy: {}/{} ({:.0f}%)\n'.format(
            correct,
            (len(test_loader) * test_loader.batch_size),
            100. * correct / (len(test_loader) * test_loader.batch_size)))

In [ ]:
# ---------------------------------------
# Train / Test the model
# ---------------------------------------
epochs = 10
for epoch in range(1, epochs + 1):
    train(net, train_loader, optimizer, epoch)
    test(net, test_loader)

In [ ]:
# ---------------------------------------
# Export the trained model
# ---------------------------------------
torch.save(net.state_dict(), 'model.pth')

In [ ]:
! ls -al model.pth